In [1]:
#this imports labels from json files, fixes coordinates, creates debugging images of the original image and the overlaid labels and cellIDs,
#then measures PCC of channel1 (fiji C2), channel 2 (fiji C3) , mean, sum intensity and area of each label and saves a s PDF

import os
import numpy as np
import json
import pandas as pd
import tifffile as tiff
import matplotlib.pyplot as plt
from skimage.draw import polygon2mask
from skimage.measure import regionprops
from scipy.stats import pearsonr

# Paths to image and mask folders; adjust with your own complete paths
IMAGE_FOLDER = r"C:\Users\picca\Documents\PhD data\coloc_tests\files"
MASK_FOLDER = r"C:\Users\picca\Documents\PhD data\coloc_tests\masks"
OUTPUT_FOLDER = r"C:\Users\picca\Documents\PhD data\coloc_tests\results"
DEBUG_FOLDER = os.path.join(OUTPUT_FOLDER, "debug")

os.makedirs(OUTPUT_FOLDER, exist_ok=True)
os.makedirs(DEBUG_FOLDER, exist_ok=True)

# Pixel to micron conversion factor (if available)
pixel_to_micron = 0.03529  # scaling taken from fiji - set measurements, reads this in automatically

# Function to compute Pearson correlation coefficient
def compute_pearson(channel1, channel2):
    valid_mask = (channel1 > 0) & (channel2 > 0)
    if np.sum(valid_mask) > 10:  # Ensure enough pixels for meaningful correlation
        return pearsonr(channel1[valid_mask].flatten(), channel2[valid_mask].flatten())[0]
    return np.nan

# Function to convert polygon coordinates to a binary mask
def polygon_to_mask(image_shape, polygon_coords):
    # Swap coordinates from (x, y) to (y, x)
    # these are read in swapped for some reason -can check with debug images
    swapped_coords = [(y, x) for x, y in polygon_coords]
    return polygon2mask(image_shape, swapped_coords)

# Process each image
for image_file in os.listdir(IMAGE_FOLDER):
    if not image_file.endswith(".tif"):
        continue

    image_path = os.path.join(IMAGE_FOLDER, image_file)
    mask_path = os.path.join(MASK_FOLDER, image_file.replace(".tif", ".json"))

    if not os.path.exists(mask_path):
        print(f"Warning: No matching mask for {image_file}")
        continue

    # Load image
    image = tiff.imread(image_path)  # Shape: (C, Y, X)

    # Load mask data
    with open(mask_path, 'r') as f:
        mask_data = json.load(f)  # Assuming JSON contains GeoJSON-like structure

    # Initialize an empty label mask
    labeled_mask = np.zeros(image.shape[1:], dtype=int)

    # Iterate over each feature and assign a unique label
    label_id = 1
    for feature in mask_data['features']:
        geometry = feature['geometry']
        if geometry['type'] == 'Polygon':
            coords = geometry['coordinates'][0]  # Extract first set of coordinates
            poly_mask = polygon_to_mask(image.shape[1:], coords)
            labeled_mask[poly_mask] = label_id
            label_id += 1
        elif geometry['type'] == 'MultiPolygon':
            for coords in geometry['coordinates']:
                poly_mask = polygon_to_mask(image.shape[1:], coords[0])
                labeled_mask[poly_mask] = label_id
                label_id += 1
        else:
            print(f"Unsupported geometry type: {geometry['type']} in {mask_path}")
            continue

    num_labels = label_id - 1
    print(f"{image_file}: Detected {num_labels} labeled regions")

    # Save debug image to visualize labeled regions
    plt.figure(figsize=(10, 10))
    plt.imshow(image[0], cmap='gray')  # Display channel 0
    plt.imshow(labeled_mask, cmap='nipy_spectral', alpha=0.5)  # Overlay mask with transparency

    # Annotate each region with its Cell_ID
    props = regionprops(labeled_mask)
    for prop in props:
        y, x = prop.centroid
        plt.text(x, y, str(prop.label), color='white', fontsize=12, ha='center', va='center')

    plt.title(f"Labeled Regions in {image_file}")
    debug_image_path = os.path.join(DEBUG_FOLDER, image_file.replace(".tif", "_overlay.tiff"))
    plt.axis('off')  # Hide axes for better visualization

    # Save the figure as a TIFF file
    plt.savefig(debug_image_path, format='tiff', bbox_inches='tight', pad_inches=0)
    plt.close()
    print(f"Saved overlay image with labels: {debug_image_path}")

    # Choose channels to compare (modify if needed)
    if len(image.shape) < 3:
        print(f"Skipping {image_file} - unexpected image shape {image.shape}")
        continue

#channel numbering starts at 0 here
    channel1 = image[1]  # First channel (fiji C2), KRAS
    channel2 = image[2]  # Second channel (fiji C3), BRAF

    results = []
    for region in regionprops(labeled_mask, intensity_image=channel1):
        cell_mask = labeled_mask == region.label
        pearson_corr = compute_pearson(channel1[cell_mask], channel2[cell_mask])

        # Calculate mean intensities for both channels within the current region
        mean_intensity_channel1 = np.mean(channel1[cell_mask])
        mean_intensity_channel2 = np.mean(channel2[cell_mask])

        # Calculate sum intensities for both channels within the current region
        sum_intensity_channel1 = np.sum(channel1[cell_mask])
        sum_intensity_channel2 = np.sum(channel2[cell_mask])

# Calculate area in pixels
        area_pixels = region.area

        # Convert area to square microns if conversion factor is available
        if pixel_to_micron:
            area_microns = area_pixels * (pixel_to_micron ** 2)
        else:
            area_microns = np.nan  # If no conversion factor, set as NaN

        # Append the results
        results.append({
            'Cell_ID': region.label,
            'Pearson_Correlation': pearson_corr,
            'Mean_Intensity_Channel1': mean_intensity_channel1,
            'Mean_Intensity_Channel2': mean_intensity_channel2,
            'Sum_Intensity_Channel1': sum_intensity_channel1,
            'Sum_Intensity_Channel2': sum_intensity_channel2,
            'Area_Pixels': area_pixels,
            'Area_Square_Microns': area_microns
        })

    # Save results to CSV
    df = pd.DataFrame(results)
    output_csv = os.path.join(OUTPUT_FOLDER, image_file.replace(".tif", ".csv"))
    df.to_csv(output_csv, index=False)
    print(f"Saved: {output_csv}")

KRASG12D_BRAF_R188L_27.tif: Detected 30 labeled regions
Saved overlay image with labels: C:\Users\picca\Documents\PhD data\coloc_tests\results_test\debug\KRASG12D_BRAF_R188L_27_overlay.tiff
Saved: C:\Users\picca\Documents\PhD data\coloc_tests\results_test\KRASG12D_BRAF_R188L_27.csv
